# **Шаг 1**: Предобработка тестового датасета

Загрузим стандартные библиотеки для работы с данными и визуализации:

In [1]:
import numpy as np                  
import pandas as pd
import random

import os
import glob

import matplotlib.pyplot as plt
## VV: работа с dicom файлами

## VV: библиотека для jpeg/dcom 
try:
    import pylibjpeg
except:
    !pip install /kaggle/input/rsna-packages/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
#! pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm
#! pip install --upgrade pydicom 

import pydicom

## VV: библиотеки для контроля прогресса и распарралеливания
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import json

## VV: библиотека для работы с изображениями
import cv2

Processing /kaggle/input/rsna-packages/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-packages/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-packages/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: pydicom
    Found existing installation: pydicom 2.3.1
    Uninstalling pydicom-2.3.1:
      Successfully uninstalled pydicom-2.3.1


Посмотрим на тестовый датасет.

In [2]:
df_test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
df_test

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


Пропишем пути к изображениям в датасет.

In [3]:
df_test['image_path'] =  '/kaggle/input/rsna-breast-cancer-detection/test_images/' + \
df_test.patient_id.map(str) + \
'/' + df_test.image_id.map(str) + '.dcm'

df_test.iloc[0]['image_path']

'/kaggle/input/rsna-breast-cancer-detection/test_images/10008/736471439.dcm'

Используем ту же предобработку, что и для тренировочного датасета.

In [4]:
## VV: адаптировано из блокнота https://www.kaggle.com/code/fabiendaniel/dicom-cropped-resized-png-jpg
def crop_image(path, show=True, width=224, height=224, resize = True, gabor = False): ## определим функцию
    dicom = pydicom.dcmread(path)               ## прочитаем dicom            
    later = dicom.ImageLaterality               ## правая или левая грудь
    img = dicom.pixel_array                     ## прочитаем само изображение
    if img.max() - img.min() != 0:
        img = (img - img.min()) / (img.max() - img.min()) ## проведем нормализацию   
    img *= 255                                  ## от диапазона 0-1 перейдем к 0-255
    img = np.uint8(img)

    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img ## VV: приведение к одному виду фотометрической интерпретации

    bin_pixels = cv2.threshold(img, 20, 255, cv2.THRESH_BINARY)[1] ## проведем бинаризацию
   
    # определим контуры на изображении
    contours, _ = cv2.findContours(bin_pixels.astype(np.uint8), \
                                   cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)

    # Create a mask from the largest contour
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
   
    # Use bitwise_and to get masked part of the original image
    out = cv2.bitwise_and(img,mask)
    
    # get bounding box of contour
    y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
    x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
    
    x1 = int(0.99 * x1)
    x2 = int(1.01 * x2)
    y1 = int(0.99 * y1)
    y2 = int(1.01 * y2)
    cropped = out[y1:y2, x1:x2]
    
    if gabor:
        ksize = 35  # The local area to evaluate
        sigma = 3.0  # Larger Values produce more edges
        lambd = 10.0
        gamma = 0.5
        psi = 0 
        theta = 0.57 ## check EDA for details
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_64F)
    
    ## VV: let's resize
    dim = (width, height)
    resized = cv2.resize(cropped, dim,interpolation = cv2.INTER_AREA)
    if later == 'R':
        resized = cv2.flip(resized, 1) ## приведем все к одной латеральности
    if show:
        if resize:
            plt.imshow(resized, cmap="turbo") 
        else:
            plt.imshow(cropped, cmap="turbo")
    if resize:
        return resized ## изменим размер
    else:
        return cropped ## или просто остановимся на вырезанном фрагменте

Сделаем предобработку изображений тестового датасета, пропишем новые пути в тестовый датасет.

In [5]:
DATASET_NAME_test = f'RSNA-cropped-png-test'
SAVE_FOLDER_test = f"/kaggle/working/{DATASET_NAME_test}"
EXTENSION = 'png'
os.makedirs(SAVE_FOLDER_test, exist_ok=True)
def process(f,save_folder="", extension="png"):
    patient = f.split('/')[-2]
    image = f.split('/')[-1][:-4]
    img = crop_image(f, gabor = True)
    cv2.imwrite(f"{SAVE_FOLDER_test}/{patient}_{image}.{extension}", img)


paths_test = df_test['image_path']

_ = Parallel(n_jobs=4)(
    delayed(process)(uid, save_folder=SAVE_FOLDER_test, extension=EXTENSION)
    for uid in tqdm(paths_test)
)
   
df_test['new_image_path'] = SAVE_FOLDER_test + '/' +\
df_test.patient_id.map(str) + '_' +\
df_test.image_id.map(str) + '.' +\
EXTENSION     

  0%|          | 0/4 [00:00<?, ?it/s]

Теперь используем функцию, чтобы предобработать изображения из тренировочного набора. Определим, папку для сохранения и формат. Для этого возьмем за основу код из блокнота dicom-cropped-resized-png-jpg.

Определим параметры для сохранения изображений и создадим папку:

In [6]:
# df_train_cut_cut
#df_train_cut['new_image_path'] = '/kaggle/input/pytorch-and-rsna-screening-mammography' + '/' +\
df_test['new_image_path'] = SAVE_FOLDER_test + '/' +\
df_test.patient_id.map(str) + '_' +\
df_test.image_id.map(str) + '.' +\
EXTENSION 

На случай, если в датасете есть пропуски по возрасту, заменим их

In [7]:
df_test['age'] = df_test['age'].fillna((df_test['age'].mean()))

# **Шаг 2:** Инференс

Импортируем библиотеки:

In [8]:
from PIL import Image
## VV: библиотеки для анализа изображений и построения нейронной сети и подгрузки данных
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [9]:
mod = torch.load('/kaggle/input/rsna-packages/vgg11_bn.pt')

Загрузим обученную модель

In [10]:
## need to generate a dummy model to be able to load a final one
## https://stackoverflow.com/questions/55488795/unpickling-saved-pytorch-model-throws-attributeerror-cant-get-attribute-net
class RSNA_model(nn.Module):
    def __init__(self,model,csv_use = False, n_neu=4,  csv_neu=2):
        super().__init__()
        self.n_neu = n_neu       
        self.csv_neu = csv_neu
        self.csv_use = csv_use
        self.features = model
        self.num_classes = model.num_classes
        self.csv = nn.Sequential(nn.Linear(self.csv_neu, self.n_neu),
                                 nn.ReLU(),
                                 nn.Linear(self.n_neu, 100),
                                 nn.ReLU())
        if self.csv_use:
            self.classif = nn.Linear(model.num_classes + 100, 2)
        else:
            self.classif = nn.Linear(model.num_classes, 2)
        
        
    def forward(self, img, meta):
        img_1 = self.features(img)
        meta_1 = self.csv(meta)
        if self.csv_use:
            x = torch.cat((img_1, meta_1), dim=1)       
        else:
            x = img_1
        x   = self.classif(x)
        return x

    
dummy_model = RSNA_model(mod)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

final_model = torch.load('/kaggle/input/rsna-breast-cancer-pt-3-model-training/best_model.pt',\
                        map_location=device)

Зафиксируем генератор случайных чисел для воспроизводимости результатов:

In [11]:
def seed_everything(seed):
    random.seed(seed) # фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # фиксируем заполнения хешей
    np.random.seed(seed) # фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # фиксируем генератор случайных чисел pytorch
    torch.cuda.manual_seed(seed) # фиксируем генератор случайных чисел для GPU
    torch.backends.cudnn.deterministic = True # выбираем только детерминированные алгоритмы (для сверток)
    torch.backends.cudnn.benchmark = False # фиксируем алгоритм вычисления сверток

seed_everything(12345)

Создадим класс для работы с датасетом. Наследуемся от класса Dataset https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset

In [12]:
## VV: адаптировано из https://www.kaggle.com/code/andradaolteanu/rsna-breast-cancer-eda-pytorch-baseline
class RSNADataset(Dataset):
    
    def __init__(self, dataframe, path_ds, csv_columns, augment = False, is_train=True):
        self.dataframe = dataframe
        self.is_train = is_train
        self.path_ds = path_ds
        self.csv_columns = csv_columns
        if augment:
            self.transform = transforms.Compose([#transforms.RandomHorizontalFlip(p = 0.5),
                                                 #transforms.RandomVerticalFlip(p = 0.3),
                                                 transforms.RandomPerspective(p=0.1),
                                                 transforms.RandomRotation(degrees=(-90,90)),
                                                 transforms.Grayscale(num_output_channels=3),
                                                 transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                                 transforms.ToTensor()])
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        self.dataframe['new_image_path'] =  self.path_ds + \
        self.dataframe.patient_id.map(str) + '_' + \
        self.dataframe.image_id.map(str) + '.png'

        image_path = self.dataframe['new_image_path'].iloc[index]
        im_frame = Image.open(image_path)
        transf_image = (self.transform(im_frame))        
        csv_data = (np.array(self.dataframe.iloc[index][self.csv_columns].values, 
                            dtype=np.float32)) #torch.FloatTensor
        if not self.is_train:
            self.dataframe = self.dataframe.replace({'R': 0, 'L' : 1})
            subm_data = (np.array(self.dataframe.iloc[index][['patient_id','laterality']].values,dtype=np.float32)) #,'laterality'
           # subm_data = csv_data #(np.array(self.dataframe.iloc[index][['patient_id']].values,dtype=np.float32))
        # Apply transforms
        
        
        if self.is_train:
            label = self.dataframe['cancer'].iloc[index]
            return transf_image, csv_data, label
           # return {"image": transf_image, 
            #        "meta": csv_data, 
            #        "target": self.dataframe['cancer'].iloc[index]}
        else:
            return transf_image, csv_data, subm_data
           # return {"image": transf_image, 
           #         "meta": csv_data}

Создадим экземпляр класса.

In [13]:
TEST_BATCH_SIZE = 40
rsna_test = RSNADataset(df_test, 
                        csv_columns = ['age','implant'], 
                        path_ds = SAVE_FOLDER_test + '/',
                        is_train=False)

test_loader = torch.utils.data.DataLoader(rsna_test, 
                                           batch_size = TEST_BATCH_SIZE,
                                           shuffle = False, 
                                           drop_last = False)


Зададим функцию для тестирования модели

In [14]:
def test(model, loader):
    with torch.no_grad():
        model.eval()
        probs = []
        prediction_id = []
        preds = []
        for k, data in enumerate(loader): # valid_loader):
            image, meta, subm_data = data
            image, meta = image.to(device), meta.to(device)
            out = model(image, meta)
            pred = out.argmax(dim=1, keepdim=True).detach().cpu().numpy()
            #print(pred)
            prob_fun = nn.Softmax(dim=1)
            prob = prob_fun(out)[:,1]
            preds.append (pred)
            probs.append(prob.flatten().detach().cpu().numpy())
            laterality = ['L' if i > 0.5 else 'R' for i in subm_data[:,1]]
            patient_id = [str(round(i)) for i in subm_data[:,0].numpy()]
            pred_id = [i[0] +'_' + i[1] for i in zip(patient_id , laterality)]
            prediction_id.append(pred_id)
    preds = [item for sublist in preds for item in sublist]        
    probs = [item for sublist in probs for item in sublist]
    prediction_id = [item for sublist in prediction_id for item in sublist]
    submission = pd.DataFrame({'prediction_id': prediction_id,
                               'cancer': probs,
                              'cancer_pred':preds})
    submission = submission.groupby('prediction_id')[['cancer']].agg('mean').reset_index()                            
    return submission

In [15]:
sub = test(final_model,test_loader)
sub['cancer'] = sub['cancer'].fillna(0)
sub['cancer'] = (sub['cancer'].values > 0.5).astype(int)
sub

,prediction_id,cancer
0,10008_L,1
1,10008_R,1


In [16]:
import shutil
shutil.rmtree(SAVE_FOLDER_test)

In [17]:
sub.to_csv('/kaggle/working/submission.csv', index=False)